# Nepal Earthquake

From: [Kaggle Nepal Earthquake](https://www.kaggle.com/datasets/imtkaggleteam/nepal-earthquake)

### Context
The Nepal Earthquake Severity Index is designed to provide an overview of estimated severity of impacts resulting from the earthquake of 25 April 2015. It is not a replacement for first hand damage and needs assessment information, but can support prioritisation during early stages of the response. It estimates severity based on: 1) the intensity of the earthquake; 2) population; 3) vulnerability of housing and population. This index will be updated to take account of: validation against first hand reports and improvements to the severity model; improved sources of data (quality, timeliness and scale); changing requirements as the response continues. 

In [3]:
import platform
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import seaborn as sns
import sklearn
from sklearn import datasets, decomposition, metrics, preprocessing, utils
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor, plot_tree
from tqdm.notebook import tqdm

### Data

In [4]:
%config InlineBackend.figure_format="retina"  # For high DPI display

sns.set_style("darkgrid")
sns.set_context("notebook")

plotly.offline.init_notebook_mode(connected=True)

tqdm.pandas()

print(sklearn.__version__)  # Version tested on sklearn.__version__ == 1.3.x

1.2.2


### Utility Functions

In [5]:
def pprint_var(**kwargs):
    for k, v in kwargs.items():
        print(f"{k} = {v:.4g}")

#### Data loading

In [7]:
DATA = Path("data")    # Change to your data folder
assert DATA.exists()

In [8]:
eq_raw_df = pd.read_csv(DATA / "nepal-earthquake-severity-index-latest.csv", low_memory=False)